# CityBikes

Send a request to CityBikes for the city of your choice. 

In [38]:
import pandas as pd
import requests

In [39]:
url = "http://api.citybik.es/v2/networks/greenbikeslc" # Salt Lake City
headers = {"Accept": "application/json"}
response = requests.get(url, headers=headers)
inner = response.json().get('network', {})
df = pd.json_normalize(inner['stations'])
df.head()

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra.uid,extra.renting,extra.returning,extra.last_updated,extra.address,extra.rental_uris.android,extra.rental_uris.ios,extra.has_ebikes,extra.ebikes,extra.normal_bikes
0,00d7f326bdd9fb9650b026408580cf98,EcoBox Station @ 580S 500W,40.75689,-111.90523,2024-12-12T04:26:44.704285Z,5,7,bcycle_greenbikeslc_4591,1,1,1733977604,580 S 500 W,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,True,1,4
1,05c8e0daae43d4192d0500a9712f000b,Energy Development Station @ 136S Main Street,40.76624,-111.89124,2024-12-12T04:26:44.704096Z,2,17,bcycle_greenbikeslc_2737,1,1,1733977604,136 S Main St,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,True,0,2
2,0b8fecfdc7edf284b4c3aad5f21ef044,City Weekly Station @ 353S Main Street,40.76142,-111.89091,2024-12-12T04:26:44.704026Z,10,13,bcycle_greenbikeslc_2255,1,1,1733977604,353 S Main St,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,True,0,10
3,0bae47c705dfdea3b51dd920286e1372,500N 300W Station,40.78045,-111.89945,2024-12-12T04:26:44.704408Z,3,15,bcycle_greenbikeslc_7607,1,1,1733977604,500 N 300 W,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,True,0,3
4,15558de8032f59cff1a3e952ee5dd003,CW Urban Station @ theBeverly,40.76816,-111.90809,2024-12-12T04:26:44.704242Z,5,11,bcycle_greenbikeslc_4527,1,1,1733977604,63 S 600 W,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,True,2,3


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         59 non-null     object 
 1   name                       59 non-null     object 
 2   latitude                   59 non-null     float64
 3   longitude                  59 non-null     float64
 4   timestamp                  59 non-null     object 
 5   free_bikes                 59 non-null     int64  
 6   empty_slots                59 non-null     int64  
 7   extra.uid                  59 non-null     object 
 8   extra.renting              59 non-null     int64  
 9   extra.returning            59 non-null     int64  
 10  extra.last_updated         59 non-null     int64  
 11  extra.address              59 non-null     object 
 12  extra.rental_uris.android  59 non-null     object 
 13  extra.rental_uris.ios      59 non-null     object 
 

In [41]:
#checking to see if there are any duplicates so this column could be potentially a primary key
df.duplicated('extra.uid').sum()

0

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [42]:
cleaned = df[['extra.uid','name', 'extra.address', 'latitude', 'longitude', 'empty_slots', 'free_bikes']]

Put your parsed results into a DataFrame.

In [43]:
cleaned = cleaned.rename(columns={'extra.address': 'address'})
cleaned.insert(0, 'short_id', cleaned['extra.uid'].str[20:]) #specfically doing [20:] instead of [-4:] in case the length of the id number ever jumps to more than 4 digits
cleaned.drop('extra.uid', axis=1, inplace=True)
cleaned.to_csv("../data/greenbikeslc.csv", index=False)
cleaned.head()

,short_id,name,address,latitude,longitude,empty_slots,free_bikes
0,4591,EcoBox Station @ 580S 500W,580 S 500 W,40.75689,-111.90523,7,5
1,2737,Energy Development Station @ 136S Main Street,136 S Main St,40.76624,-111.89124,17,2
2,2255,City Weekly Station @ 353S Main Street,353 S Main St,40.76142,-111.89091,13,10
3,7607,500N 300W Station,500 N 300 W,40.78045,-111.89945,15,3
4,4527,CW Urban Station @ theBeverly,63 S 600 W,40.76816,-111.90809,11,5
